In [27]:
import warnings
warnings.filterwarnings(action='ignore')

In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [29]:
path='./heart.csv'

In [30]:
df=pd.read_csv(path)

In [31]:
df.columns

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'],
      dtype='object')

In [32]:
#finding and removing the duplicate columns

In [33]:
duplicateRowsDF = df[df.duplicated(keep='first')]
print("Duplicate Rows except first occurrence based on all columns are :")
duplicateRowsDF

Duplicate Rows except first occurrence based on all columns are :


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
164,38,1,2,138,175,0,1,173,0,0.0,2,4,2,1


In [34]:
df.drop_duplicates(keep='first',inplace=True)

In [35]:
columns=list(df.columns)

In [36]:
df[(df.age>=20)&(df.age<40)].target.mean()

0.7333333333333333

In [37]:
df[(df.age>=40)&(df.age<65)].target.mean()

0.5365853658536586

In [38]:
df[(df.age>=65)].target.mean()

0.5121951219512195

In [39]:
def replaceAgeWithCatogery(value):
    if(value>=20 and value<40):
        return 'YoungAdult'
    elif(value>=40 and value<65):
        return 'MiddleageAdult'
    else:
        return 'Oldaged'



In [40]:
df.age=df.age.apply(replaceAgeWithCatogery)

In [41]:
def replaceSexWithCatogery(value):
    if(value==1):
        return 'male'
    else: 
        return 'female'


In [42]:
df.sex=df.sex.apply(replaceSexWithCatogery)

In [43]:
df.columns

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'],
      dtype='object')

In [44]:
def getCatogericalandNumericalColumns(columns):
    numericalcolumns=[]
    catogericalcolumns=[]
    for column in columns:
        if(len(df[column].value_counts())<=7):
            catogericalcolumns.append(column)
        else:
            numericalcolumns.append(column)
    return numericalcolumns,catogericalcolumns

In [45]:
(num,cat)=getCatogericalandNumericalColumns(columns)

In [46]:
num

['trestbps', 'chol', 'thalach', 'oldpeak']

In [47]:
#checking for outliers

In [3]:
fig, (axes) = plt.subplots(nrows=1, ncols=4, figsize=(16, 5))
sns.boxplot(data=df[['trestbps']],  palette='Set2', ax=axes[0]);
sns.boxplot(data=df[['chol']],  palette='Set2', ax=axes[1]);
sns.boxplot(data=df[['thalach']],  palette='Set2', ax=axes[2]);
sns.boxplot(data=df[['oldpeak']],  palette='Set2', ax=axes[3]);

NameError: name 'plt' is not defined

In [49]:
Q1 = df[num].quantile(0.25)
Q3 = df[num].quantile(0.75)
IQR = Q3 - Q1

In [50]:
for col in num:
    low  = Q1[col] - 1.5 * IQR[col]
    high = Q3[col] + 1.5 * IQR[col]
# Filtering Values between Q1-1.5IQR and Q3+1.5IQR
    query_string = '{} < @low or {} > @high'.format(col, col)

    outlier_arr1 = df.query(query_string).index
    print(outlier_arr1)

Int64Index([8, 101, 110, 203, 223, 241, 248, 260, 266], dtype='int64')
Int64Index([28, 85, 96, 220, 246], dtype='int64')
Int64Index([272], dtype='int64')
Int64Index([101, 204, 221, 250, 291], dtype='int64')


In [51]:
def updateNumLowHigh(col):
    low  = Q1[col] - 1.5 * IQR[col]
    high = Q3[col] + 1.5 * IQR[col]
    df.loc[df[col]<low,[col]]=low
    df.loc[df[col]>high,[col]]=high

In [52]:
for col in num:
    updateNumLowHigh(col)

In [54]:
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,MiddleageAdult,male,3,145.0,233.0,1,0,150.0,0,2.3,0,0,1,1
1,YoungAdult,male,2,130.0,250.0,0,1,187.0,0,3.5,0,0,2,1
2,MiddleageAdult,female,1,130.0,204.0,0,0,172.0,0,1.4,2,0,2,1
3,MiddleageAdult,male,1,120.0,236.0,0,1,178.0,0,0.8,2,0,2,1
4,MiddleageAdult,female,0,120.0,354.0,0,1,163.0,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,MiddleageAdult,female,0,140.0,241.0,0,1,123.0,1,0.2,1,0,3,0
299,MiddleageAdult,male,3,110.0,264.0,0,1,132.0,0,1.2,1,0,3,0
300,Oldaged,male,0,144.0,193.0,1,1,141.0,0,3.4,1,2,3,0
301,MiddleageAdult,male,0,130.0,131.0,0,1,115.0,1,1.2,1,1,3,0


In [60]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer

In [62]:
cat.remove('target')

In [64]:
preprocess = make_column_transformer(
    (StandardScaler(), num),
    (OneHotEncoder(),  cat)
)

In [65]:
newdf=preprocess.fit_transform(df)

In [69]:
newdf=pd.DataFrame(data=newdf)

In [71]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [72]:
X_train,X_test,y_train,y_test=train_test_split(newdf,target,test_size=30,random_state=1)

In [73]:
rfc = RandomForestClassifier()     #by default random forest is using 100 trees
rfc.fit(X_train,y_train)

y_pred=rfc.predict(X_test)

print(accuracy_score(y_test,y_pred))

0.8333333333333334


In [84]:
r=RandomForestClassifier(n_estimators=100,criterion='gini',max_depth=2,random_state=7)     #by default criterion is gini
r.fit(X_train,y_train)
y_pred=r.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.8666666666666667


In [ ]:
#fitting knn 

In [85]:
# import the ML algorithm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor

from sklearn.neighbors import NearestNeighbors

In [100]:
# instantiate the classifier 
clf = KNeighborsClassifier(n_neighbors=7)  
clf

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=7, p=2,
                     weights='uniform')

In [101]:
clf.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=7, p=2,
                     weights='uniform')

In [102]:
y_pred=clf.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.8666666666666667
